In [1]:
import gym
import numpy as np
import copy, random, time, subprocess, os
from tensorflow.keras import layers, models

/opt/conda/lib/python3.7/site-packages/flatbuffers/compat.py:19: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp
/opt/conda/lib/python3.7/site-packages/keras_preprocessing/image/utils.py:23: DeprecationWarning: NEAREST is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.NEAREST or Dither.NONE instead.
  'nearest': pil_image.NEAREST,
/opt/conda/lib/python3.7/site-packages/keras_preprocessing/image/utils.py:24: DeprecationWarning: BILINEAR is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.BILINEAR instead.
  'bilinear': pil_image.BILINEAR,
/opt/conda/lib/python3.7/site-packages/keras_preprocessing/image/utils.py:25: DeprecationWarning: BICUBIC is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.BICUBIC instead.
  'bicubic': pil_image.BICUBIC,
/opt/conda/lib/python3.7/site-packages/keras_preprocessing/image/utils.py:28: Depreca

In [2]:
class QValue:
    def __init__(self):
        self.model = None

    def get_action(self, state):
        states = []
        actions = []
        for a in range(5):
            states.append(np.array(state))
            action_onehot = np.zeros(5)
            action_onehot[a] = 1
            actions.append(action_onehot)
  
        q_values = self.model.predict([np.array(states), np.array(actions)])
        optimal_action = np.argmax(q_values)
        return optimal_action, q_values[optimal_action][0]

In [3]:
def join_frames(o0, o1):
    return np.r_[o0.transpose(), o1.transpose()].transpose() 

In [4]:
q_value = QValue()

In [5]:
import datetime 
import imageio

def create_gif(checkpoint, model='model01', epsilon=0):

    BUCKET = 'gs://etsuji-car-racing-v2-{}'.format(model)
    filename = 'car-racing-v2-{}-{}.hd5'.format(checkpoint, model)
    if not os.path.isfile(filename):
        subprocess.run(['gsutil', 'cp', '{}/{}/{}'.format(BUCKET, model, filename), './'])   
    print('load model {}'.format(filename))
    q_value.model = models.load_model(filename)

    env = gym.make("CarRacing-v2", continuous=False)
    o0 = env.reset()
    o1 = copy.deepcopy(o0)
    done = 0
    total_r = 0
    c = 0

    frames = []

    while not done:
        if c % 1 == 0:
            if np.random.random() < epsilon:
                a = np.random.randint(5)
            else:
                a, _ = q_value.get_action(join_frames(o0, o1))          
        o_new, r, done, i = env.step(a)
        total_r += r
        o0, o1 = o1, o_new 
        c += 1
        frame = env.render('rgb_array')
        frames.append(frame)
        if c % 30 == 0:
            print('{}:{}'.format(a, int(total_r)), end=', ')
    print('{}:{}'.format(a, int(total_r)))
        
    now = datetime.datetime.now()
    imageio.mimsave('car-racing-v2-{}-{:05d}-{}-{}.gif'.format(
        model, int(checkpoint), int(total_r), now.strftime('%Y%m%d-%H%M%S')),
                    frames, 'GIF' , **{'duration': 1.0/30.0})

In [6]:
MODEL_NAME = 'model04'
checkpoints = [486]

for c in checkpoints:    
    create_gif(c, MODEL_NAME, epsilon=0)

AccessDeniedException: 403 324928435771-compute@developer.gserviceaccount.com does not have storage.objects.list access to the Google Cloud Storage bucket.


load model car-racing-v2-486-model04.hd5


OSError: No file or directory found at car-racing-v2-486-model04.hd5